# 在PostgreSQL中进行分子结构搜索与查询

# 一、环境搭建
本文以windows为例进行操作和演示。假设都已经安装好了postgresql，下面依次进行用户切换、启动服务端、创建数据库、导入数据、连接数据库、加载插件、函数测试。在windows上安装带rdkit插件的postgresql以及基本操作可以参考同系列文章
- 在cmd中切换到postgres用户
```cmd
runas /user:postgres cmd
```


- 在postgres用户下，启动服务端，指定数据库位置
```cmd
postgres -D d:\postgresql\data
```


- 在新cmd终端中创建molecules数据库
```cmd
createdb molecules
```


- 使用python创建info表格，加入id、smiles、LOGP等列，并导入1000条分子数据。可以参考[postgresql_by_psycopg2.ipynb]中的介绍。
- 导入完成数据后，连接进入数据库
```cmd
psql molecules
```


- 在“molecules=#”提示符下，查看刚才创建的info表
```sql
select column_name from information_schema.columns WHERE table_name ='info';
```


- 为当前数据库加载rdkit插件
```sql
create extension rdkit;
```


- 测试下rdkit插件，能正常输出结果表明环境准备成功
```sql
select id, mol_from_smiles(smiles::cstring) mol_col from info limit 5;
```

# 二、数据表准备
数据库、rdkit插件都没问题后，接下来依次新建schema、转换mol对象、建立数据表。
- 新建schema，命名为rdk。schema相当于一个命名空间，允许多个用户建立各自的schema来同时操作数据库，每个schema下的对象名字不能重复，但不同的schema中可以重复。如果不指定schema时，默认在public下操作。
```sql
create schema rdk;
```


- （删除名为rdk的schema）
```sql
drop schema rdk cascade;
```


- 查看有哪些schema
```sql
select nspname from pg_namespace;
```


- 将SMILES转换为mol对象，列名记为m，并连同id一起保存在rdk下的mols表中
- 其中mol_from_smiles(smi_col::cstring)是rdkit函数，能够将smi_col列作为字符串读取并转换为mol对象
```sql
select * into rdk.mols from (select id, mol_from_smiles(smiles::cstring) m from info) tmp where m is not null;
```


- 查看rdk下的数据表，相当于mysql的show databases
```sql
select table_name from information_schema.tables WHERE table_schema = 'rdk';
```


- 对rdk.mols表中的m列建立索引，命名为molidx
```sql
create index molidx on rdk.mols using gist(m);
```


- 将rdk.mols表中的id列设为主键，主键必须唯一且非空
```sql
alter table rdk.mols add primary key (id);
```

# 三、结构搜索与查询
终于进入正题了，准备俩小时，查询五分钟。
## 1.smiles子结构搜索
- 查看多少分子包含了苯环，顺便介绍一下子结构搜索的操作符
- "@>"含义：判断该符号右边的结构是不是该符号左边结构的子结构
- "<@"含义：判断该符号左边的结构是不是该符号右边结构的子结构
- "@="含义：判断该符号左右两边的结构是否相同

```sql
select count(*) from rdk.mols where m@>'c1ccccc1';
```


- 查看前5条包含了吲哚结构的分子
```sql
select id, m from rdk.mols where m@>'C1(NC=C2)=C2C=CC=C1' limit 5;
```

## 2.smarts子结构搜索
- 查看多少分子包含了苯环，在smarts后需要加上"::qmol"。smarts查询一般会比smiles慢一些。
```sql
select count(*) from rdk.mols where m@>'c1[o,s]ncn1'::qmol;
```

## 3.立体信息的结构搜索
- 在搜索时，无论在查询smiles或smarts中是否加入了手性符号，都是不考虑立体结构的。想设置立体结构匹配可以将do_chiral_sss设置为true
```sql
set rdkit.do_chiral_sss=true;
```


- 再进行立体结构的搜索
```sql
select * from rdk.mols where m@>'NC(=O)[C@@H]1CCCN1C=O' limit 10;
```

## 4.带取代基的结构搜索
- 可以使用mol_adjust_query_properties()函数，对搜索结构进行更精确的控制。
- 该函数作用
1. 将"*"匹配为任意原子
2. 对环中的原子加入连接度的匹配，也就是只能匹配明确的环结构或指定了取代基的环结构
3. 匹配芳香性

- 使用该函数配合smiles和通配符"*"，可以搜索具有特定位取代基（侧链）的子结构。以搜索2,6-双取代吡啶为例
```sql
select id, m from rdk.mols where m@>mol_adjust_query_properties('*c1cccc(*)n1') limit 10;
```


- 也可以通过JSON格式的参数来停用连接度匹配，这样，除了指定位置必须含有取代基外，其他位置也可以含有取代基
```sql
select id, m from rdk.mols where m@>mol_adjust_query_properties('*c1cccc(*)n1', '{"adjustDegree":false}') limit 10;
```


- 还可以通过其他的参数参数进行更精细的匹配，这里就不深入讨论了，感兴趣可以研究下下方链接里的内容。

本文参考自[rdkit官方文档](http://www.rdkit.org/docs/Cartridge.html)。